In [2]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# Hardcode username and password
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.H2(children='Danica Hesemann'),
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.A([
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height' : '10%',
                    'width' : '10%',
                    'float' : 'right',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
                })
    ], href='https://www.snhu.edu/'),
    # Place radio items at top
    html.Div(
        dcc.RadioItems(
            id = 'filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'WAT'},
                {'label': 'Mountain Rescue', 'value': 'MTN'},
                {'label': 'Disaster Rescue', 'value': 'DST'},
                {'label': 'Reset', 'value': 'RST'}
            ],
            value='RST'
        )   
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} 
            for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table 
        #to make it user-friendly for your client, like sorting and paging
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    html.Br(),
    html.Hr(),   
    
    #This sets up the dashboard so that your chart and your 
    #geolocation chart are side-by-side
    html.Div(className='row',
        style={'display' : 'flex'},
            children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### Add code to filter interactive data table with MongoDB queries
    dff = df
    if filter_type == 'RST':
        dff = pd.DataFrame.from_records(shelter.read({}))
    elif filter_type == 'WAT':
        dff = pd.DataFrame(list(shelter.read(
            {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", 
                               "Newfoundland"]}, "sex_upon_outcome": "Intact Female", 
             "age_upon_outcome_in_weeks" : {"$gte" : 26, "$lte" : 156}})))
    elif filter_type == 'MTN':
        dff = pd.DataFrame(list(shelter.read(
            {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", 
                               "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
             "sex_upon_outcome": "Intact Male", 
             "age_upon_outcome_in_weeks" : {"$gte" : 26, "$lte" : 156}})))
    elif filter_type == 'DST':
        dff = pd.DataFrame(list(shelter.read(
            {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", 
                               "Golden Retriever", "Bloodhound", "Rottweiler"]}, 
             "sex_upon_outcome": "Intact Male", 
             "age_upon_outcome_in_weeks" : {"$gte" : 20, "$lte" : 300}})))
            
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} 
             for i in dff.columns]
    data=dff.to_dict('records')


    return (data,columns)
        
 
# Highlights selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Create pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(
        data_frame = dff,
        names = 'breed'
    )
    return [
        dcc.Graph(style={'width': '700px', 'height': '500px'},            
            figure = fig
        )    
    ]

   
# Create geolocation chart
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    # Code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '700px', 'height': '500px'}, 
               center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app